# This notebook implements of K-means algorithm

In [17]:
import numpy as np
import pandas as pd
import norgatedata 
import datetime
import pickle
import matplotlib.pyplot as plt

from scipy.stats import percentileofscore
import warnings

warnings.filterwarnings('ignore')

%matplotlib inline 

In [87]:
import seaborn as sns
import random



from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples


#similar to above but now without correlation matrix output but the dataframe
def clusterKMeansBase1(df,maxNumClusters=10,n_init=10,initn=2):
    
    """
    KMeans Clustering with randomized centroid intializations and searching for optimal K with silouette score
   
    Parameters
    ----------
    df : DataFrame
        Observations to run k means consisting of factors as features
   
    Returns
    -------
    clstrs: dictionary
        Dictionary of key cluster index and value as list of symbols
        
    silh: Series
        Silouette score for each asset
    """
    x,silh=df.copy(),pd.Series()# observations matrix
    for init in range(n_init):
        for i in range(initn,maxNumClusters+1):
            kmeans_=KMeans(n_clusters=i,n_jobs=1,n_init=1,max_iter=300,random_state=0)
            kmeans_=kmeans_.fit(x)
            silh_=silhouette_samples(x,kmeans_.labels_)
            stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:
                silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
    
    clstrs={i:df.index[np.where(kmeans.labels_==i)[0]].tolist() \
    for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    return clstrs,silh



def return_centroid_ranges_mid(df):
    """
    Random centroid initializations generator: size(9,3)
   
    Parameters
    ----------
    df : DataFrame
        Observations to run k means consisting of factors as features
   
    Returns
    -------
    init_array : numpy array
        Numpy array of size 9 x 3 containing centroid initializations
    """

    
    init_array = np.array([[50,50,50],
         [random.uniform(65, 85),random.uniform(15,35),random.uniform(65,85)],
         [random.uniform(65,85),random.uniform(65,85),random.uniform(15,35)],
         [random.uniform(65,85),random.uniform(15,35),random.uniform(15,35)],
         [random.uniform(15,35),random.uniform(65,85),random.uniform(15,35)],
         [random.uniform(15,35),random.uniform(65,85),random.uniform(65,85)],
         [random.uniform(15,35),random.uniform(15,35),random.uniform(65,85)],
         [random.uniform(65,85),random.uniform(65,85),random.uniform(65,85)],
         [random.uniform(15,35),random.uniform(15,35),random.uniform(15,35)]])
    
    
    return init_array

def clusterKMeansBase_3fact_init(df,iteration=100):
    
    """
    KMeans Clustering with fixed K=9 and 3 factors.
   
    Parameters
    ----------
    df : DataFrame
        Observations to run k means consisting of factors as features
    iteration : int
        Amount of iterations for looping with different initiliazations 
   
    Returns
    -------
    clstrs: dictionary
        Dictionary of key cluster index and value as list of symbols
        
    silh: Series
        Silouette score for each asset
    """
    
   
    x,silh=df.copy(),pd.Series()# observations matrix
    
    for it in range(iteration):
        init_array = return_centroid_ranges_mid2(df)
        kmeans_=KMeans(n_clusters=9,n_jobs=-1,n_init=1,max_iter=300,init=init_array)
        kmeans_=kmeans_.fit(x)
        silh_=silhouette_samples(x,kmeans_.labels_)
        stat=(silh_.mean()/silh_.std(),silh.mean()/silh.std())
        if np.isnan(stat[1]) or stat[0]>stat[1]:
            silh,kmeans=silh_,kmeans_
    newIdx=np.argsort(kmeans.labels_)
   
    clstrs={i:df.index[np.where(kmeans.labels_==i)[0]].tolist() \
    for i in np.unique(kmeans.labels_) } # cluster members
    silh=pd.Series(silh,index=x.index)
    
    return clstrs,silh


In [24]:
def return_factor_df(i,f = ['qua','val','vol']):
    """
    Returns a dataframe from a single date's observation with 3 factors(features) and constieunts as index
   
    Parameters
    ----------
    i: int
        ith index of the monthly observation dataframe
    f: list
        list of factors of length 3
   
    Returns
    -------
    factors_df: dataframe
        Dataframe of 3 features and index as constituents within a given ith observation
        
    """

    #find out dates
    date = str(month_const.index[i])[:10]
    #switch through iloc and loc. 
    # this part is for handling missing values and only selecting non null stocks and rank it by avg volume
    month_val_i = month_const.iloc[i] 
    top_1000_volume = volume_const.iloc[i][month_val_i[month_val_i==1].index].sort_values(ascending=False).iloc[:1000].index

    null1 = list(factor_df.loc[(date,'mom')][top_1000_volume].isnull()[factor_df.loc[(date,'mom')][top_1000_volume].isnull() == True].index)
    null2 = list(factor_df.loc[(date,'vol')][top_1000_volume].isnull()[factor_df.loc[(date,'vol')][top_1000_volume].isnull() == True].index)
    null3 = list(factor_df.loc[(date,'val')][top_1000_volume].isnull()[factor_df.loc[(date,'val')][top_1000_volume].isnull() == True].index)
    null4 = list(factor_df.loc[(date,'qua')][top_1000_volume].isnull()[factor_df.loc[(date,'qua')][top_1000_volume].isnull() == True].index)

    null_list = set(null1+null2+null3+null4)
    include_list = [x for x in top_1000_volume if x not in null_list]
    
    #filter out factor values from filtered stock lists
    factors = factor_df.loc[date].droplevel(0).loc[f][include_list]
    
    
    
    
    
    factors_df = factors.transpose()


    for col in factors_df.columns:
        s = factors_df[col]
        factors_df[col] = s.apply(lambda x: percentileofscore(s, x))

        
    return factors_df

In [41]:
def run_kmeans_prep(month_const,volume_const,factor_df,f = ['qua','val','vol']):
    
    """
    Run K-means pipeline from data preprocessing. This uses clusterKMeansBase1 to run K-means
   
    Parameters
    ----------
    month_const : DataFrame
        Monthly observations of constituent time series
        
    volume_const: DataFrame
        Volume time series
        
    factor_df: DataFrame
        Daily Observations containing all constieunts and all factors 
    
    f: list
        List of factors with length 3
   
    Returns
    -------

    clusters_list: list
        Containing a time series of cluster information indicating which symbols are in which clusters 
    """
    
    #determine length of our rebalance observations
    length = month_const.shape[0]
    
    #define lists
    
    clusters_list = []
    
    
    #loop through observations
    
    for i in range(22,30):
        #find out dates
        date = str(month_const.index[i])[:10]
        #switch through iloc and loc. 
        # this part is for handling missing values and only selecting non null stocks and rank it by avg volume
        month_val_i = month_const.iloc[i] 
        top_1000_volume = volume_const.iloc[i][month_val_i[month_val_i==1].index].sort_values(ascending=False).iloc[:1000].index

        null_list = []
        for fact in f:
            temp_null = list(factor_df.loc[(date,fact)][top_1000_volume].isnull()[factor_df.loc[(date,fact)][top_1000_volume].isnull() == True].index)
            null_list += temp_null

        null_list = set(null_list)
        include_list = [x for x in top_1000_volume if x not in null_list]
        
        #filter out factor values from filtered stock lists
        factors = factor_df.loc[date].droplevel(0).loc[f][include_list]
    
    
    
    
        factors_df = factors.transpose()

        
    
        for col in factors_df.columns:
            s = factors_df[col]
            factors_df[col] = s.apply(lambda x: percentileofscore(s, x))
        
        # run kmeans and append
        clstrs,silh=clusterKMeansBase1(factors_df)
        
        
        clusters_list.append(clstrs)
        
        print(i)
        
    return clusters_list

In [88]:
def run_kmeans_prep_3fact(month_const,volume_const,factor_df,f = ['qua','mom','vol']):
    
    """
    Run K-means pipeline from data preprocessing. This uses clusterKMeansBase_3fact_init to run K-means
   
    Parameters
    ----------
    month_const : DataFrame
        Monthly observations of constituent time series
        
    volume_const: DataFrame
        Volume time series
        
    factor_df: DataFrame
        Daily Observations containing all constieunts and all factors 
    
    f: list
        List of factors with length 3
   
    Returns
    -------

    clusters_list: list
        Containing a time series of cluster information indicating which symbols are in which clusters 
    """
    
    #determine length of our rebalance observations
    length = month_const.shape[0]
    
    #define lists
    
    clusters_list = []
    
    
    #loop through observations
    
    for i in range(length):
        #find out dates
        date = str(month_const.index[i])[:10]
        #switch through iloc and loc. 
        # this part is for handling missing values and only selecting non null stocks and rank it by avg volume
        month_val_i = month_const.iloc[i] 
        top_1000_volume = volume_const.iloc[i][month_val_i[month_val_i==1].index].sort_values(ascending=False).iloc[:1000].index

        null_list = []
        for fact in f:
            temp_null = list(factor_df.loc[(date,fact)][top_1000_volume].isnull()[factor_df.loc[(date,fact)][top_1000_volume].isnull() == True].index)
            null_list += temp_null

        null_list = set(null_list)
        include_list = [x for x in top_1000_volume if x not in null_list]
        
        #filter out factor values from filtered stock lists
        factors = factor_df.loc[date].droplevel(0).loc[f][include_list]
    
    
    
    
        factors_df = factors.transpose()

        for col in factors_df.columns:
            s = factors_df[col]
            factors_df[col] = s.apply(lambda x: percentileofscore(s, x))
        
        # run kmeans and append
        clstrs,silh=clusterKMeansBase_3fact_init(factors_df,iteration = 300)
        
        
        clusters_list.append(clstrs)
        
        print(i)
        
    return clusters_list

In [10]:
path = 'D:/Udacity/final/'

Importing volume, constieunts, and all factors dataframe

In [11]:
russell_volume_avg200 = pd.read_csv(path + 'russell_volume_avg200.csv',parse_dates=True,index_col = 0 )
russell_constituent = pd.read_csv(path + 'russell_constituent_pre_f.csv',parse_dates=True,index_col = 0)
factor_df = pd.read_csv(path + 'factor_df_f.csv',parse_dates=True,index_col = [0,1])

We select values after 2003 to align with our factors df

In [12]:
russell_volume_avg200 = russell_volume_avg200.loc['2003-01-02':]
russell_constituent = russell_constituent.loc['2003-01-02':]

Filter out end of month dates, since we plan to rebalance on the end of month

In [13]:
dateRange = []  
tempYear = None  
tempTradeDays = russell_volume_avg200.index
dictYears = tempTradeDays.groupby(tempTradeDays.year)
for yr in dictYears.keys():
    tempYear = pd.DatetimeIndex(dictYears[yr]).groupby(pd.DatetimeIndex(dictYears[yr]).month)
    for m in tempYear.keys():
        dateRange.append(max(tempYear[m]))
dateRange = pd.DatetimeIndex(dateRange).sort_values()

In [14]:
month_const = russell_constituent.reindex(dateRange)
volume_const = russell_volume_avg200.reindex(dateRange)

Initial Solution: Optimize clusters by optimal 'K' with double looping and random initializations.

In [53]:
clstr_info = run_kmeans_prep(month_const,volume_const,factor_df,f=['qua','vol','val'])

22
23
24
25
26
27
28
29


In [51]:

f1 = 'qua'
f2 = 'vol'
f3 = 'val'
ii = 4

clstrs = clstr_info[ii]
DF = return_factor_df(ii,f=['qua','vol','val'])

PLOT = go.Figure()

for C in range(len(clstrs)):
    
    PLOT.add_trace(go.Scatter3d(x = DF.reindex(clstrs[C])[f1],
                                y = DF.reindex(clstrs[C])[f2],
                                z = DF.reindex(clstrs[C])[f3],
                                mode = 'markers', marker_size = 8, marker_line_width = 1,
                                name = 'Cluster ' + str(C)))
    
    
PLOT.update_layout(width = 600, height = 600, autosize = True, showlegend = True,
               scene = dict(xaxis=dict(title = f1 , titlefont_color = 'black'),
                            yaxis=dict(title = f2 , titlefont_color = 'black'),
                            zaxis=dict(title = f3 , titlefont_color = 'black')),
               font = dict(family = "Gilroy", color  = 'black', size = 12))

What we do here is we cluster by qua, vol, and val. We solved two problems. First if we let k means algo find the optimal k by random centroid initializations, we will end up atmost 2 or 3 clusters and we are unable to evaluate which clusters are significant. Second, we want to be able to fix the cluster index on a consistent characteristic of the clusters. IF we randomize the centroids, then as we backtest, we will see that clstr 1 at a point in time has  high vol and high val characteristics, and later in time clstr1 turns out to have low vol and low val. We want evaluate clearly on which cluster and what scale of qua, vol, and val factors can we analyze to determine the assets to invest. What we did is we strictly choose k = 9 (3 features with 9 sectors), and then explicitly tell where to initialize the centroids. We intialize the centroids as randomizing locations around the midpoints (0.25 to 0.75) of the sectors. 

In [89]:
clstr_info_k9 = run_kmeans_prep_3fact(month_const,volume_const,factor_df,f=['qua','vol','val'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212


In [63]:
# cluster = pickle.load(open(path+'clstr_info_k9_1.pkl','rb'))

In [92]:
pickle.dump(clstr_info_k9, open(path+'clstr_info_k9_f.pkl','wb'))

In [20]:
import plotly.graph_objects as go

Plotting clusters

In [96]:
#131 shows cluster 1
f1 = 'qua'
f2 = 'vol'
f3 = 'val'
ii = 100

clstrs = clstr_info_k9 [ii]
DF = return_factor_df(ii,f=['qua','vol','val'])

PLOT = go.Figure()

for C in range(len(clstrs)):
    
    PLOT.add_trace(go.Scatter3d(x = DF.reindex(clstrs[C])[f1],
                                y = DF.reindex(clstrs[C])[f2],
                                z = DF.reindex(clstrs[C])[f3],
                                mode = 'markers', marker_size = 8, marker_line_width = 1,
                                name = 'Cluster ' + str(C)))
    
    
PLOT.update_layout(width = 600, height = 600, autosize = True, showlegend = True,
               scene = dict(xaxis=dict(title = f1 , titlefont_color = 'black'),
                            yaxis=dict(title = f2 , titlefont_color = 'black'),
                            zaxis=dict(title = f3 , titlefont_color = 'black')),
               font = dict(family = "Gilroy", color  = 'black', size = 12))

==============================================================================

========================================================================

Time for constructing our cluster time series, based on rebalance days. Since the length of the clstr_info_k9 is already equal to rebalance days (observations) we can just use that to form a list comprehension and then create a series of 1 and 0's similar to constituent series, but now the 1 and 0s indicate investment decisions. 

In [28]:
#create symbol watchlist
symbol_watchlist = list(month_const.columns)

In [97]:
#dictionary to store each cluster timeseries dataframe investment index
clstr_td_ct = {}
# loop through each clusters
for i in range(9):
    # get a list of lists, each observation is a shape (n_observations,n_symbols in cluster at point observation)
    # and contains symbol information in cluster
    clstr_k = [clstr[i] for clstr in clstr_info_k9]
    clstr_ts = []
    # loop through each observation's symbols of the cluster
    for clstr_info in clstr_k:
        
        # compare with our watchlist, if symbols in watch list is in observation symbols we say 1 (since we want to
        # invest in it)
        ts_temp = [1 if sym in clstr_info else 0 for sym in symbol_watchlist]
        clstr_ts.append(ts_temp)
    
    #create a dataframe for each cluster (still in monthly observations)
    temp_df = pd.DataFrame(data = np.array(clstr_ts), index= month_const.index, columns =  month_const.columns)
    # we reindex based on the full time series and then ffill to prepare for backtesting
    clstr_td_ct[i] = temp_df.reindex(russell_constituent.index).ffill()

In [98]:
import pickle
# pickle.dump(clstr_td_ct, open(path + 'qvv_clstr_ts_1.pkl','wb'))
pickle.dump(clstr_td_ct, open(path + 'qvv_clstr_ts_f.pkl','wb'))